In [1]:
## ------------------------------------------ packages ------------------------------------------ ##
import sys
import redis
import paho.mqtt.client as mqtt
import json


## ------------------------------------- initial variables -------------------------------------- ##
REDIS_HOST = 'redis-19224.c293.eu-central-1-1.ec2.cloud.redislabs.com'
REDIS_PORT = 19224
REDIS_USERNAME = 'default'
REDIS_PASSWORD = 'XT6DLlALIOibGPPIrPXZk1cYe6NGca32'

MQTT_HOST = 'mqtt.eclipseprojects.io'
MQTT_PORT = 1883
MQTT_TOPIC = 's308563'


## -------------------------------------- redis connection -------------------------------------- ##
print('Redis connection status:')
try:
    redis_client = redis.Redis(host=REDIS_HOST, port=REDIS_PORT, username=REDIS_USERNAME, password=REDIS_PASSWORD)
    is_connected = redis_client.ping()
    print(f'  -- connection result \u2192 {is_connected}')
except:
    print(f'  -- connection result \u2192 False')
    sys.exit()


## -------------------------------------- mqtt connection --------------------------------------- ##
# functions
def on_connect(mqtt_client, userdata, flags, rc):
    print('Message broker connection status:')
    mqtt_client.subscribe(MQTT_TOPIC)
    print(f'  -- subscription result \u2192 {str(rc)}')
    print('Log:')

def on_message(mqtt_client, userdata, msg):
    if msg.topic == MQTT_TOPIC:
        decoded_message = json.loads(msg.payload.decode())

        battery_ts_name = decoded_message['mac_address']+':battery'
        power_ts_name =  decoded_message['mac_address']+':power'
        if redis_client.exists(battery_ts_name) == 0:
            redis_client.ts().create(battery_ts_name)
            print(f'  -- the `{battery_ts_name}` time-series was created')
        if redis_client.exists(power_ts_name) == 0:
            redis_client.ts().create(power_ts_name)
            print(f'  -- the `{power_ts_name}` time-series was created')

        for event in decoded_message['events']:
            timestamp_in_ms = event['timestamp']
            battery_level = int(event['battery_level'])
            power_plugged = int(event['power_plugged'])
            redis_client.ts().add(battery_ts_name, timestamp_in_ms, battery_level)
            redis_client.ts().add(power_ts_name, timestamp_in_ms, power_plugged)
        print(f'  -- {len(decoded_message["events"])} events were uploaded to `{battery_ts_name}` and `{power_ts_name}` time-series')

    else:
        print('  -- message broker error!')

# creating an mqtt client
mqtt_client = mqtt.Client()
mqtt_client.on_connect = on_connect
mqtt_client.on_message = on_message
mqtt_client.connect(MQTT_HOST, MQTT_PORT)


## ----------------------------------------- main loop ------------------------------------------ ##
mqtt_client.loop_forever()

Redis connection status:
  -- connection result → True
Message broker connection status:
  -- subscription result → 0
Log:
  -- the `0x86266b5e53b:battery` time-series was created
  -- the `0x86266b5e53b:power` time-series was created
  -- 10 events were uploaded to `0x86266b5e53b:battery` and `0x86266b5e53b:power` time-series
  -- 10 events were uploaded to `0x86266b5e53b:battery` and `0x86266b5e53b:power` time-series
  -- 10 events were uploaded to `0x86266b5e53b:battery` and `0x86266b5e53b:power` time-series
  -- 10 events were uploaded to `0x86266b5e53b:battery` and `0x86266b5e53b:power` time-series
  -- 10 events were uploaded to `0x86266b5e53b:battery` and `0x86266b5e53b:power` time-series
  -- 10 events were uploaded to `0x86266b5e53b:battery` and `0x86266b5e53b:power` time-series
  -- 10 events were uploaded to `0x86266b5e53b:battery` and `0x86266b5e53b:power` time-series
  -- 10 events were uploaded to `0x86266b5e53b:battery` and `0x86266b5e53b:power` time-series
  -- the `0xf

KeyboardInterrupt: 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c66c7ed3-adef-44a1-955f-b8ce9193a2a0' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>